In [1]:
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
import joblib
from pathlib import Path

# --- MLflow & Sklearn Imports ---
import mlflow
from mlflow.models.signature import infer_signature # <--- CORRECCIÓN 1: Importar esto
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# --- Configuración de Rutas (Ajustado a tu estructura) ---
# Si el cuaderno está en 'Proyecto/Notebooks', la raíz es el padre (..)
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
data_path = project_root / "data" / "maternal_risk_processed.csv"

# Añadir la carpeta 'app' al path para importar config si es necesario
# (Asumiendo que config.py está en 'app/' o en la raíz)
sys.path.append(str(project_root))
sys.path.append(str(project_root / "app"))

# Intentamos importar tu manager, si falla usamos mlflow directo
try:
    from app.mlflow_manager import mlflow_manager # Ajusta según donde tengas este archivo
    print("✅ MLflow Manager importado correctamente.")
except ImportError:
    # Fallback si no encuentra el módulo, configuramos mlflow básico aquí
    print("⚠️ No se encontró mlflow_manager, usando configuración manual.")
    mlflow.set_tracking_uri(f"sqlite:///{project_root}/mlflow.db")
    mlflow.set_experiment("Maternal_Health_Project")
    class MockManager:
        def start_run(self, run_name): return mlflow.start_run(run_name=run_name)
    mlflow_manager = MockManager()

# --- 1. CARGA DE DATOS ---
print(f"🔄 Cargando datos desde: {data_path}")
if not data_path.exists():
    raise FileNotFoundError(f"No se encuentra el archivo: {data_path}")

df = pd.read_csv(data_path)
print(f"✅ Dataset cargado: {df.shape}")

X = df.drop(['RiskLevel', 'Risk_Num'], axis=1)
y = df['Risk_Num']

# --- 2. PREPROCESAMIENTO (SCALING) ---
print("\n🔄 Escalando datos...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# --- 3. DEFINICIÓN DEL MODELO ---
def create_model(input_shape):
    model = Sequential([
        Dense(16, activation='relu', input_shape=(input_shape,), name="base_layer_1"),
        Dense(8, activation='relu', name="base_layer_2"),
        Dropout(0.1),
        Dense(3, activation='softmax', name="output_layer")
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

model = create_model(X_train.shape[1])
print("✅ Modelo creado.")

# --- 4. ENTRENAMIENTO CON MLFLOW (CORREGIDO) ---
print("\n🔄 Iniciando entrenamiento y registro en MLflow...")

# Parámetros a registrar
params = {
    "epochs": 50,
    "batch_size": 16,
    "layers": [16, 8, 3],
    "dropout": 0.1
}

with mlflow_manager.start_run(run_name="Base_Model_Training") as run:
    # 1. Loguear parámetros
    mlflow.log_params(params)
    
    # 2. Entrenar
    history = model.fit(
        X_train, y_train,
        epochs=params["epochs"],
        batch_size=params["batch_size"],
        validation_split=0.1,
        verbose=1
    )
    
    # 3. Métricas finales
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    mlflow.log_metrics({"test_accuracy": accuracy, "test_loss": loss})
    print(f"📊 Accuracy en Test: {accuracy:.4f}")

    # --- CORRECCIONES CLAVE AQUI ---
    
    # A) INFERIR FIRMA (Signature)
    # Esto le dice a MLflow qué tipo de datos espera el modelo (evita el warning de signature)
    print("✍️ Generando firma del modelo...")
    # Usamos una muestra de predicción para inferir la salida
    predictions = model.predict(X_train[:5])
    signature = infer_signature(X_train, predictions)

    # B) GUARDAR SCALER COMO ARTEFACTO
    # Guardamos el scaler localmente primero y luego lo subimos al Run
    print("📦 Guardando Scaler en MLflow...")
    joblib.dump(scaler, "scaler.pkl")
    mlflow.log_artifact("scaler.pkl") 
    # Ahora el scaler viaja con el modelo, no importa dónde muevas la API.

    # C) GUARDAR MODELO CON FIRMA
    # Usamos mlflow.tensorflow.log_model directamente para asegurar compatibilidad
    print("💾 Guardando modelo...")
    mlflow.tensorflow.log_model(
        model, 
        artifact_path="model",  # Nombre de la carpeta dentro del run
        signature=signature     # Adjuntamos la firma
    )

    print("\n✅ ¡Entrenamiento completado y guardado correctamente!")
    print(f"🆔 Run ID: {run.info.run_id}")

⚠️ No se encontró mlflow_manager, usando configuración manual.


2026/02/01 19:35:33 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/01 19:35:33 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/01 19:35:33 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/01 19:35:33 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/01 19:35:33 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/01 19:35:33 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/01 19:35:33 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/01 19:35:33 INFO alembic.runtime.migration: Will assume non-transactional DDL.
c:\Users\andre\OneDrive\Desktop\Proyecto\venv2\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model inste

🔄 Cargando datos desde: c:\Users\andre\OneDrive\Desktop\Proyecto\data\maternal_risk_processed.csv
✅ Dataset cargado: (1014, 8)

🔄 Escalando datos...
✅ Modelo creado.

🔄 Iniciando entrenamiento y registro en MLflow...
Epoch 1/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3320 - loss: 1.2225 - val_accuracy: 0.3293 - val_loss: 1.1651
Epoch 2/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3471 - loss: 1.0809 - val_accuracy: 0.4146 - val_loss: 1.0536
Epoch 3/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4390 - loss: 0.9972 - val_accuracy: 0.5122 - val_loss: 0.9884
Epoch 4/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5391 - loss: 0.9322 - val_accuracy: 0.6098 - val_loss: 0.9455
Epoch 5/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5734 - loss: 0.8915 - val_accuracy: 0.6098 - val_loss: 0.9094
Epoch 6/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5802 - loss: 0.8646 - val_accuracy: 0.6341 - val_loss: 0.8757
Epoch 7/50
46/46 ━━━━━━━━━━

2026/02/01 19:35:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


📦 Guardando Scaler en MLflow...
💾 Guardando modelo...

✅ ¡Entrenamiento completado y guardado correctamente!
🆔 Run ID: 5518f0299d9c4c749e4111bf55637ee6
